Before beginning, we need to download database and pictures as well as install sqlite kernel into jupyter. Assuming conda is installed, run the following bash commands:

```shell
conda create -n 'sqlmm' ipython jupyter 
conda activate sqlmm
conda install xeus-sqlite -c conda-forge

curl https://mystery.knightlab.com/schema.png -o schema.png
curl https://github.com/NUKnightLab/sql-mysteries/raw/master/sql-murder-mystery.db -o sql-murder-mystery.db
```

See
- [SQL Murder Mystery interactive homepage](https://mystery.knightlab.com/)
- [SQL Murder Mystery github page](https://github.com/NUKnightLab/sql-mysteries)
- [Xues SQLite kernel for Jupyter Notebooks](https://blog.jupyter.org/a-jupyter-kernel-for-sqlite-9549c5dcf551)

In [1]:
%LOAD sql-murder-mystery.db

# SQL Murder Mystery Prompt

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a *murder* that occurred sometime on *Jan.15, 2018* and that it took place in *SQL City*. Start by retrieving the corresponding crime scene report from the police department’s database. 

![schema.png](schema.png)

### We know that ...

- date **20180115**
- city **SQL City**
- type **murder**

## Crime report

In [2]:
SELECT * 
FROM crime_scene_report
WHERE date = '20180115'
  AND type = 'murder'
  AND city = 'SQL City'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


### We know that ...

Witness 1
- last house on **Northwestern Dr**

Witness 2
- Name **Annabel**
- Lives on **Franklin Ave**

## Witness transcripts

In [3]:
WITH witness1 as (
  SELECT *
  FROM person
  WHERE 
      address_street_name = "Franklin Ave"
  AND name like '%Annabel%'),
witness2 as (
  SELECT *
  FROM person
  WHERE address_street_name = "Northwestern Dr"
  ORDER BY address_number DESC
  LIMIT 1),
witness_id as (
  SELECT id FROM witness1
  UNION
  SELECT id FROM witness2
)

SELECT *
FROM interview AS i
JOIN witness_id AS w
ON i.person_id = w.id

person_id,transcript,id
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",14887
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",16371


## We know that ...

Suspect 1
- membership number starts with **48Z**
- is **gold** member
- Licens plate include **H42W**

Suspect 2
- was at gym on **20180109**

We don't know if suspects are the same person, but let's assume so.

## Suspect interview

In [4]:
WITH members_on_date_of_interest as (
  SELECT DISTINCT membership_id 
  FROM get_fit_now_check_in
  WHERE check_in_date = '20180109'
)

SELECT p.name, transcript
FROM get_fit_now_member AS m
JOIN members_on_date_of_interest AS c
  ON c.membership_id = m.id 
JOIN interview AS i
  ON p.id = i.person_id
JOIN drivers_license AS d
  ON p.license_id = d.id
JOIN person AS p
  ON m.person_id  = p.id
WHERE
      m.id like '48Z%'
  AND m.membership_status = 'gold'
  AND d.plate_number like '%H42W%'

name,transcript
Jeremy Bowers,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


## What we know so far

- is **female**
- is rich
- between **65"** and **67**
- **red** hair
- car is **Tesla Model S**
- Attended **SQL Symphony Concert** 3 times in **201712**

In [5]:
WITH suspects_in_drivers_license AS (
    SELECT id
    FROM drivers_license
    WHERE
              hair_color = 'red'
          AND gender = 'female'
          AND car_make = 'Tesla'
          AND car_model = 'Model S'
          AND height BETWEEN 65 AND 67
), suspects_in_fb_event AS (
    SELECT 
      person_id, 
      count() as event_count
    FROM facebook_event_checkin
    WHERE 
          event_name = 'SQL Symphony Concert'
      AND date BETWEEN 20171200 AND 20171233
    GROUP BY person_id
    HAVING event_count = 3
)

SELECT p.name
FROM person AS p
JOIN suspects_in_fb_event AS e ON p.id = e.person_id
JOIN suspects_in_drivers_license AS d ON p.license_id = d.id

name
Miranda Priestly
